In [2]:
# Import libraries
import os
from PIL import Image
import numpy as np

from scipy import stats
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score

Load data

In [3]:
# Read classes
classes = sorted(os.listdir("Data"))

# Initialize train and test data list
train_images_path, test_images_path = [], []
train_labels, test_labels = [], []

# Set training data ratio
train_ratio = 0.7

for i, class_name in enumerate(classes):

    # Read images in class folder
    class_data = os.listdir(os.path.join("Data", class_name))

    # Train data are the first images
    for img_name in class_data[:int(train_ratio*len(class_data))]: #0.7 primeres fotos
        # Get only images paths
        train_images_path.append(os.path.join("Data", class_name, img_name))
        # Append labels
        train_labels.append(i)

    # Test data are the last images
    for img_name in class_data[int(train_ratio*len(class_data)):]:
        # Get only images paths
        test_images_path.append(os.path.join("Data", class_name, img_name))
        #Append labels
        test_labels.append(i)


train_images_path=np.array(train_images_path)
test_images_path=np.array(test_images_path)
train_labels = np.array(train_labels)
test_labels=np.array(test_labels)

#print(train_images_path,train_images_path.shape)
#print(train_labels)

Conv NN

In [4]:
import torch
from torch.utils.data import DataLoader, Dataset as BaseDataset

class Dataset(BaseDataset):

    """
    Inherit from torch Dataset class. We overwrite _len_ and _getitem_ methods
    """

    def __init__(self, data, labels, num_classes):

        # Call parent init
        BaseDataset.__init__(self)

        # Save raw data
        self.data = data
        self.labels = labels

        # Transform labels into one-hot encoding
        self.hot_labels = torch.zeros(len(data), num_classes, dtype=float)
        for idx, label in enumerate(labels):
            self.hot_labels[idx, label] = 1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        # Load image
        img = np.array(Image.open(self.data[index]))
        
        torch_image = torch.from_numpy(img)

        # Cast to float and normalize it
        torch_image = torch_image.type(torch.float)/255 - 0.5


        # Permute image into shape (channel, width, height)
        torch_image = torch_image.permute((2, 0, 1))

        return torch_image, self.hot_labels[index]

train_dataset = Dataset(train_images_path, train_labels, len(classes))
test_dataset = Dataset(test_images_path, test_labels, len(classes))

    # Set batch-size, train_loader and test_loader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

# Iterate through the DataLoader
for images, labels in train_loader:
    # Debug: Print batch shape and type
    print(f"Batch Image Shape: {images.shape}")
    print(f"Batch Label Shape: {labels.shape}")
    break  

# print("Train image paths", len(train_images_path))
# print("Test image paths", len(test_images_path))

Batch Image Shape: torch.Size([16, 3, 256, 256])
Batch Label Shape: torch.Size([16, 4])


In [5]:
from PIL import Image
import torch
import torch.nn as nn

class ConvNet(nn.Module):

    """
    Set custom small convolutional nn.
    """

    def __init__(self, input_size, num_classes):

        # Call parent's init
        super(ConvNet, self).__init__()

        # Define backbone
        self.conv_layers = nn.Sequential(
            nn.Conv2d(input_size, 500, (5, 5)),
            nn.MaxPool2d(3),
            nn.Conv2d(500, 250, (3, 3)),
            nn.MaxPool2d(3),
            nn.Conv2d(250, 100, (7, 7)),
            nn.ReLU(),
            nn.Conv2d(100, 250, (5, 5)),
            nn.ReLU(),
            nn.Conv2d(250, 100, (3, 3)),
            nn.ReLU(),
            nn.Conv2d(100, 10, (3, 3)),
            nn.ReLU()
        )
        

        # Set classification layer
        self.fc = nn.Linear(1690, num_classes)

    def forward(self, x):

        batch_size = x.shape[0]

        # Apply backbone
        y = self.conv_layers(x)

        # Apply classification layer
        y = y.view(batch_size, -1)
        return self.fc(y)


In [6]:
from tqdm import tqdm

device = "cpu" #"cuda" on colab
print("classes", classes)

TRAIN = False
if TRAIN:

    # Num epochs
    num_epochs = 25

    # Initialie model class with random weights
    model = ConvNet(3, len(classes)).to(device)

    # Set loss function, optimizer and learning-rate scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs)

    # Set initialize validation score
    valid_score = 0

    # Train the model
    for epoch in range(num_epochs):
        model.train()

        print("Epoch: [", epoch + 1, "/", num_epochs, "] | learning rate", scheduler.get_lr())

        for images, labels in tqdm(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # print("epoch", epoch, "loss", loss.item(), end="\r")
        scheduler.step()

        # Test the model
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in tqdm(test_loader):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == torch.max(labels, 1)[1]).sum().item()
            print('Test Accuracy: %.4f %%' % (100 * correct / total))
        if correct/total > valid_score:
            print("saving model...")
            torch.save(model, "drive/MyDrive/emma_erasmus/model.pt")
            valid_score = correct/total
else:
    model = torch.load("model1.pt", map_location=device)
    # Test the model
    correct = 0
    total = 0
    predictions = []
    ground_truth = []
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)

            predicted_ = predicted.cpu().item()
            label_ = torch.max(labels, 1)[1].cpu().item()

            predictions.append(predicted_)
            ground_truth.append(label_)

            correct += predicted_ == label_
        print('\nTest Accuracy: %.4f %%' % (100 * correct / total))

    # Confusion matrix
    c_matrix = confusion_matrix(ground_truth, predictions)
    print("Confusion matrix", c_matrix)

    



classes ['glioma_tumor', 'meningioma_tumor', 'normal', 'pituitary_tumor']


100%|██████████| 931/931 [11:10<00:00,  1.39it/s]



Test Accuracy: 81.7401 %
Confusion matrix [[222  35  10   4]
 [ 41 204   5  24]
 [  5  13 109   5]
 [  7  19   2 226]]


ValueError: Found input variables with inconsistent numbers of samples: [931, 1]

In [8]:
print("Accuracy =", accuracy_score(ground_truth,predictions))
print("Precision =", precision_score(ground_truth,predictions,average='macro'))
print("Recall =", recall_score(ground_truth,predictions,average='macro'))

Accuracy = 0.8174006444683136
Precision = 0.8244266231535604
Recall = 0.8198087736530772
